In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os

In [5]:
# load the candidate records
for candidate_years in os.listdir("CRP_IDs"):
    candidate_csv = pd.read_csv(f"CRP_IDs/{candidate_years}").dropna()
    print(candidate_years)
    for row in candidate_csv.itertuples():
        mydb = mysql.connector.connect(
        host="localhost",
        port=3308,
        user="root", 
        password="secret",
        database="ftvBackEnd"
        )
        cursor = mydb.cursor()
        query = """
        INSERT IGNORE INTO
            candidate (`name`, cid, party, distIdRunFor, FECCandID, cycle)
        VALUES
            (%s,%s,%s,%s,%s,%s)
            
        """
        cursor.execute(query, (row.CRPName, row.CID, row.Party, row.DistIDRunFor, row.FECCandID, candidate_years.split(".")[0]))
        mydb.commit()
        mydb.close()
candidate_csv

2018.csv
2020.csv
2022.csv


,CID,CRPName,Party,DistIDRunFor,FECCandID
0,N00047923,"Aazami, Shervin",D,CA30,H2CA30291
1,N00048268,"Abdelhamid, Rana",D,NY12,H2NY12197
2,N00048100,"Abud, Edward",D,FLS2,S2FL00482
3,N00048254,"Acciard, Julian",R,NH01,H2NH01254
4,N00048052,"Acheson, Phelan",I,UT02,H2UT02431
...,...,...,...,...,...
1328,N00047850,"Zhou, Lily",R,CAS1,S2CA00948
1329,N00048172,"Zimmerman, Marina",R,CO03,H2CO03260
1330,N00047893,"Zink, Jeff",R,AZ07,H2AZ07187
1331,N00035616,"Zinke, Ryan",R,MT02,H2MT02050


In [32]:
# load the top ten contributors per candidate per cycle
top_ten_funding_data = pd.read_csv("scraped_2020_top_ten.csv", index_col=[0])
# drop those with holes
top_ten_funding_data = top_ten_funding_data[~top_ten_funding_data["cand_name"].isnull()]
insert_industry_q = """
INSERT IGNORE INTO 
    industry(`code`, name)
VALUES
    (%s, %s)
"""
insert_funding_q = """
INSERT INTO
    `candidate-industry`(candidate_id, industry_id, indivs, pacs, total, last_updated)
VALUES
    (
    (SELECT id FROM candidate WHERE cid=%s AND cycle=%s),
    (SELECT id FROM industry WHERE `code`=%s),
    %s,
    %s,
    %s,
    %s
    )
"""
for i, row in top_ten_funding_data.iterrows():
    mydb = mysql.connector.connect(
        host="localhost",
        port=3308,
        user="root", 
        password="secret",
        database="ftvBackEnd"
        )
    cursor = mydb.cursor()
    candidate_name = row.cand_name
    cid = row.cid
    cycle = int(row.cycle)
    last_updated = row.last_updated
    for i in range(10):
        industry_code = row[f"industry_code{i}"]
        industry_name = row[f"industry_name{i}"]
        indivs = row[f"indivs{i}"]
        pacs = row[f"pacs{i}"]
        total = row[f"total{i}"]
        cursor.execute(insert_industry_q, (industry_code, industry_name))
        try:
            cursor.execute(insert_funding_q, (cid, cycle, industry_code, indivs, pacs, total, last_updated))
            mydb.commit()
        except:
            print(candidate_name)
            mydb.rollback()
            
    mydb.close()   

Lamar Alexander
Lamar Alexander
Lamar Alexander
Lamar Alexander
Lamar Alexander
Lamar Alexander
Lamar Alexander
Lamar Alexander
Lamar Alexander
Lamar Alexander
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
Tammy Baldwin
John A Barrasso
John A Barrasso
John A Barrasso
John A Barrasso
John A Barrasso
John A Barrasso
John A Barrasso
John A Barrasso
John A Barrasso
John A Barrasso
Michael Bennet
Michael Bennet
Michael Bennet
Michael Bennet
Michael Bennet
Michael Bennet
Michael Bennet
Michael Bennet
Michael Bennet
Michael Bennet
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Marsha Blackburn
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Richard Blumenthal
Roy Blunt
Roy Blunt
Roy Blunt


In [29]:
candidate_name

'Lamar Alexander'